# **Projeto da Disciplina CAP394 - Introdução à Data Science**

1. Professores:
  * Rafael Santos
  * Gilberto Ribeiro de Queiroz


2. Aluno:
  * Henrique Eduardo de Macedo
  
### Sobre o Conjunto de dados

Os dados a serem analisados são o conjunto de registros obtidos a partir do movimento dos veículos da Uber em suas viagens pela cidade de São Paulo. O dados podem ser obtidos no site https://movement.uber.com/cities.

O dataset possui 9 colunas contendo basicamente um inteiro como código da origem e do destino a hora os tempos médios do deslocamento.

Como conjunto auxiliar de dados, será utilizado uma tabela json contendo as áreas da cidade e seus códigos correspondentes também disponível no site.


### Perguntas básicas a serem respondidas:

1. Qual o melhor horário para efetuar deslocamentos para o aeroporto de Guarulhos ? <font color="red"> __RESPONDIDA__ </font>
2. No horário de pico, qual o sentido de deslocamento predominante do tráfego ?
3. É possível estimar o tempo de delocamento entre dois bairros com algum modelo de apreendizagem de máquina ?
4. É possível estimar o aumento no tempo de deslocamento devido a alguma interrupção no tráfego ?

<br>
<br>






In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r'C:\Users\hp\Downloads\uber\sao_paulo-od_zones_2017-2018-4-All-HourlyAggregate.csv')

In [3]:
df.shape

(2950937, 7)

### Sobre a lista de variáveis

  * __sourceid:__ Código da região de partida da viagem
  * __dstid:__ Código da região de destino da viagem
  * __hod:__ Hora do dia
  * __mean_travel_time:__ Tempo médio da viagem
  * __standard_deviation_travel_time:__ Desvio padrão
  * __geometric_mean_travel_time__: Média geométrica
  * __geometric_standard_deviation_travel_time__: Desvio padrão da média geométrica

In [4]:
df.columns

Index(['sourceid', 'dstid', 'hod', 'mean_travel_time',
       'standard_deviation_travel_time', 'geometric_mean_travel_time',
       'geometric_standard_deviation_travel_time'],
      dtype='object')

In [5]:
df.head()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
0,84,119,1,1700.07,556.19,1631.09,1.31
1,418,456,5,1405.84,445.70,1346.89,1.33
2,463,99,20,2481.23,288.07,2464.21,1.13
3,72,239,1,1416.94,238.94,1398.21,1.17
4,37,70,1,1794.60,482.13,1736.27,1.29


### Como traduzir o sourceid e dstid:

A fonte dos dados também fornece um json com os atributos relativos a código.

Como exemplo, para o código 0:
"NumeroZona":1,
"NomeZona":"Sé",
"NumDistrit":80,
"NomeDistri":"Sé",
"NumeroMuni":36,
"NomeMunici":"São Paulo",
"Paulo","MOVEMENT_ID":"1","DISPLAY_NAME":"Sé"
"geometry":{"type":"Polygon","coordinates":[[[-46.6291313,-23.5503351],[-46.6276154,-23.5518791],[-46.6275214,-23.5522241],[-46.6285723,-23.5521811],[-46.6295655,-23.5524881],[-46.6331403,-23.5524752],[-46.6353474,-23.55149],[-46.6354194,-23.5511471],[-46.6380082,-23.5499721],[-46.6388124,-23.549088],[-46.6388743,-23.548648],[-46.6352436,-23.5429958],[-46.6343978,-23.5438421],[-46.6338618,-23.5440981],[-46.6341168,-23.5444601],[-46.6336778,-23.5451841],[-46.6332584,-23.5474111],[-46.6322882,-23.5463551],[-46.6310282,-23.5491991],[-46.6304198,-23.5487961],[-46.6300858,-23.5494561],[-46.6291313,-23.5503351]]]}}

      



#### Pergunta 01:
Qual o melhor dia/horário para efetuar deslocamentos para os aeroportos ?

Para responder essa pergunta vamos inicialmente retirar do dataframe as colunas com com dados que não iremos usar. O dataset resultante também irá servir para os questionamentos seguintes.

após análise, excluímos as colunas abaixo com comando df.drop:
1. 'standard_deviation_travel_time'
2. 'geometric_mean_travel_time'
3. 'geometric_standard_deviation_travel_time'

In [6]:
df = df.drop(['standard_deviation_travel_time', 'geometric_mean_travel_time',
       'geometric_standard_deviation_travel_time'],axis=1)

A partir deste dataframe criamos outro, DF_GRU, com todas as viagens com dstid=374, Aeroporto Internacional de Guarulhos.

In [7]:
df_gru = df.loc[(df['dstid'] == 374)] #374 é o código do aeroporto de guarulhos
df_gru.head()

,sourceid,dstid,hod,mean_travel_time
104,162,374,22,1048.84
328,154,374,0,1590.82
402,422,374,23,2972.89
629,245,374,8,2893.96
701,383,374,14,728.90


Após agrupar as linhas conforme os seus respectivos horários e extraindo a média, temos o resultado abaixo:

In [10]:
grouped = df_gru[['mean_travel_time']].groupby(df_gru['hod'])
grouped.mean().sort_values(by='mean_travel_time',ascending=False)

,mean_travel_time
hod,
16,3724.025952
17,3573.893348
15,3500.565882
18,3280.338013
14,3204.440852
19,2947.892181
13,2937.003742
6,2895.338858
7,2816.832721


#### Resultado

Considerando todas as regiões da cidade, o horário com tempo de viagem mais lento é o das 16hs. O oposto é o horário da meia noite.

#### Pergunta 02:
No horário de pico, qual o sentido de deslocamento predominante do tráfego ?

Para resoler esse problema vamos descobrir quais horários específicos apresentam o maior volume de viagem (tráfego) para refinir qual será o nosso "horário de pico".